In [60]:
import pandas as pd
import numpy as np
import os
import json
import gc

from dataclasses import dataclass
from typing import Optional, Dict, Tuple
from pathlib import Path
from tqdm import tqdm

static_vars = ["age", "sex", "height", "weight"]

dynamic_vars = ["alb", "alp", "alt", "ast", "be", "bicar", "bili", "bili_dir",
                  "bnd", "bun", "ca", "cai", "ck", "ckmb", "cl", "crea", "crp", 
                  "dbp", "fgn", "fio2", "glu", "hgb", "hr", "inr_pt", "k", "lact",
                  "lymph", "map", "mch", "mchc", "mcv", "methb", "mg", "na", "neut", 
                  "o2sat", "pco2", "ph", "phos", "plt", "po2", "ptt", "resp", "sbp", 
                  "temp", "tnt", "urine", "wbc"]

lab_vars = ["alb", "alp", "alt", "ast", "be", "bicar", "bili", "bili_dir",
                  "bnd", "bun", "ca", "cai", "ck", "ckmb", "cl", "crea", "crp", 
                  "fgn", "fio2", "glu", "hgb", "inr_pt", "k", "lact",
                  "lymph", "mch", "mchc", "mcv", "methb", "mg", "na", "neut", 
                  "pco2", "ph", "phos", "plt", "po2", "ptt", "tnt", "wbc"]

vital_cols = ["dbp", "sbp", "map", "hr", "o2sat", "resp", "temp"]

# Original Data Load..
MIMIC_ROOT_DIR = Path('/Users/korea/EHRTTA/data/miiv')
EICU_ROOT_DIR = Path('/Users/korea/EHRTTA/data/eicu')
HIRID_ROOT_DIR = Path('/Users/korea/EHRTTA/data/hirid')

In [61]:
outcome_df = pd.read_csv(EICU_ROOT_DIR/'outcome_df.csv.gz', compression='gzip')

outcome_df

,hospitalid,subject_id,hadm_id,stay_id,hospitaladmitoffset,hospitaldischargeoffset,unitdischargeoffset,age,gender,unittype,hospitaladmitsource,ethnicity,admissionheight,admissionweight,hospitaldischargestatus,unitdischargestatus,los,mortality_inunit,los_reg
0,63,002-10034,141169,157016,-3331,7503,4172,23,Female,Med-Surg ICU,Floor,Caucasian,162.6,63.5,Alive,Alive,2.897222,0,45
1,71,002-10050,183274,211144,-67,9764,5539,67,Female,Med-Surg ICU,Operating Room,Caucasian,160.0,86.2,Alive,Alive,3.846528,0,68
2,71,002-10050,190893,221005,-2140,9912,1897,68,Female,Med-Surg ICU,Floor,Caucasian,162.6,88.0,Alive,Alive,1.317361,0,7
3,73,002-10052,137239,151900,-23,6378,3460,66,Female,MICU,Emergency Department,Caucasian,165.1,86.8,Alive,Alive,2.402778,0,33
4,73,002-10063,189145,218742,-2,7804,5180,69,Female,Neuro ICU,Emergency Department,Caucasian,152.4,68.0,Alive,Alive,3.597222,0,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78618,458,035-9957,2741786,3351785,-370,5343,2208,74,Male,Cardiac ICU,Emergency Department,Caucasian,182.9,75.7,Alive,Alive,1.533333,0,12
78619,458,035-9959,2731423,3340321,0,3873,3873,44,Male,Cardiac ICU,Direct Admit,Caucasian,185.4,130.6,Alive,Alive,2.689583,0,40
78620,458,035-996,2736458,3345874,-73,11470,2761,55,Male,Cardiac ICU,Emergency Department,African American,190.5,165.9,Alive,Alive,1.917361,0,22
78621,458,035-9966,2742533,3352628,-261,3206,1872,60,Male,MICU,Recovery Room,African American,170.1,120.2,Alive,Alive,1.300000,0,7


In [64]:
outcome_df[['stay_id', 'mortality_inunit']]

,stay_id,mortality_inunit
0,157016,0
1,211144,0
2,221005,0
3,151900,0
4,218742,0
...,...,...
78618,3351785,0
78619,3340321,0
78620,3345874,0
78621,3352628,0


In [11]:
for dir in [MIMIC_ROOT_DIR, EICU_ROOT_DIR, HIRID_ROOT_DIR]:
    # Data Load
    dynamics_df = pd.read_csv(dir/'dynamics_df.csv.gz', compression='gzip')
    static_df = pd.read_csv(dir/'static_df.csv.gz', compression='gzip')

    
    mapping = {'anchor_age' : 'age', 'gender' : 'sex'}
    static_df['var_name'] = static_df['var_name'].replace(mapping)

    with open('/Users/korea/EHRTTA/data/concept-dict.json', 'r') as f:
        concept_dict = json.load(f)

    print(f'======={dir}=======')
    print(f'총 {static_df.stay_id.nunique()} 명')
    
    observed_ts_df = dynamics_df[dynamics_df['var_name'].isin(vital_cols)].groupby(['stay_id', 'var_name'],as_index=False).count()
    display(observed_ts_df.pivot(columns='var_name', index='stay_id', values='value').describe())
    display(observed_ts_df.pivot(columns='var_name', index='stay_id', values='value').isna().sum())

    gc.collect()

=======\Users\korea\EHRTTA\data\miiv=======
총 62638 명


var_name,dbp,hr,map,o2sat,resp,sbp,temp
count,62297.000000,62563.000000,62544.000000,62593.000000,62467.000000,62300.000000,60728.000000
mean,27.371077,28.176254,27.993988,29.737654,28.525237,27.377448,8.256784
std,9.072069,8.180203,8.926030,8.367161,8.792303,9.074107,5.434505
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,24.000000,24.000000,24.000000,26.000000,25.000000,24.000000,6.000000
50%,25.000000,26.000000,25.000000,28.000000,26.000000,25.000000,7.000000
75%,30.000000,30.000000,30.000000,32.000000,30.000000,30.000000,8.000000
max,308.000000,313.000000,309.000000,324.000000,688.000000,308.000000,118.000000


var_name
dbp       296
hr         30
map        49
o2sat       0
resp      126
sbp       293
temp     1865
dtype: int64

C:\Users\korea\AppData\Local\Temp\ipykernel_13912\1715452613.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  static_df = pd.read_csv(dir/'static_df.csv.gz', compression='gzip')


=======\Users\korea\EHRTTA\data\eicu=======
총 78623 명


var_name,dbp,hr,map,o2sat,resp,sbp,temp
count,20545.000000,77322.000000,77285.000000,76927.000000,70776.000000,20558.000000,8842.000000
mean,218.195814,273.854543,109.133079,254.599126,263.968973,218.246376,216.750622
std,79.088619,28.337627,106.450391,54.606222,45.927062,79.176983,78.954987
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,181.000000,273.000000,35.000000,250.000000,264.000000,181.000000,180.000000
50%,251.000000,284.000000,63.000000,276.000000,281.000000,251.000000,246.000000
75%,281.000000,287.000000,139.000000,286.000000,287.000000,282.000000,280.000000
max,289.000000,289.000000,1614.000000,308.000000,289.000000,289.000000,289.000000


var_name
dbp      57068
hr         291
map        328
o2sat      686
resp      6837
sbp      57055
temp     68771
dtype: int64

=======\Users\korea\EHRTTA\data\hirid=======
총 13109 명


var_name,dbp,hr,map,o2sat,resp,sbp,temp
count,12760.000000,13048.000000,13047.000000,13054.000000,13041.000000,12760.000000,7825.000000
mean,708.626019,750.200874,700.468767,725.577448,564.045472,710.264577,183.852396
std,223.529453,217.444321,237.772883,210.051193,496.798978,223.628975,260.204839
min,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,1.000000
25%,643.000000,677.000000,641.000000,646.000000,17.000000,645.000000,5.000000
50%,699.000000,715.000000,702.000000,698.000000,553.000000,701.000000,10.000000
75%,717.000000,722.000000,719.000000,721.000000,973.000000,719.000000,425.000000
max,1958.000000,1965.000000,1963.000000,1955.000000,1492.000000,1962.000000,753.000000


var_name
dbp       297
hr          9
map        10
o2sat       3
resp       16
sbp       297
temp     5232
dtype: int64

In [36]:
import argparse

@dataclass
class Config:
    data_path = '/Users/korea/EHRTTA/data'
    var_info_path = '/Users/korea/EHRTTA/data/concept-dict.json'
    data_source = 'miiv'
    task_label = 'mortality_inunit'
    pid_col = 'stay_id'
    offset_col = 'charttime'

args = Config

args.data_path


'/Users/korea/EHRTTA/data'

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class ISTS_EHR_Dataset(Dataset):
    def __init__(self, args):

        super(ISTS_EHR_Dataset, self).__init__()
        
        self.args = args
        self.data_source = args.data_source
        self.task_label = args.task_label
        self.pid = args.pid_col
        self.offset = args.offset_col

        # Set the variable groups
        self.dg_cols = ["age", "sex", "height", "weight"] 
        self.ts_cols = ["dbp", "sbp", "map", "hr", "o2sat", "resp", "temp"]
        self.lab_cols = ["alb", "alp", "alt", "ast", "be", "bicar", "bili", "bili_dir","bnd", "bun", "ca", "cai", 
                         "ck", "ckmb", "cl", "crea", "crp", "fgn", "fio2", "glu", "hgb", "inr_pt", "k", "lact",
                         "lymph", "mch", "mchc", "mcv", "methb", "mg", "na", "neut", "pco2", "ph", "phos", "plt", 
                         "po2", "ptt", "tnt", "wbc"]
        self.output_cols = ["urine"]
        
        # load dataframe
        dynamics_df = pd.read_csv(Path(args.data_path)/args.data_source/'dynamics_df.csv.gz', compression='gzip')
        
        self.static_df = pd.read_csv(Path(args.data_path)/args.data_source/'static_df.csv.gz', compression='gzip')
        self.outcome_df = pd.read_csv(Path(args.data_path)/args.data_source/'outcome_df.csv.gz', compression='gzip')
        self.ts_df = dynamics_df[dynamics_df['var_name'].isin(self.ts_cols)]
        self.text_df = dynamics_df[dynamics_df['var_name'].isin([self.lab_cols + self.output_cols])]

        del dynamics_df
        
        y_data = self.outcome_df.sort_values(self.pid)[self.task_label].reset_index(drop=True)
        
        display(y_data)

        self.X_ts_data = torch.from_numpy(self.reshape_data_matrix()).float() # (N, T, C)
        # self.X_text_data =  
        self.y_data = torch.from_numpy(y_data).long()  # (N)

        print(f'Input X, Y Shape : {self.X_data.shape}, {self.y_data.shape}')
        print(f'Unique values of the label: {np.unique(y_data)}')
        
        # For univariate time series
        if len(self.X_data.shape) < 3:
            self.X_data = self.X_data.unsqueeze(2)

        self.len = self.X_data.shape[0]

    def reshape_data_matrix(self):

        df = self.ts_df.sort_values([self.pid, self.offset]).reset_index(drop=True)
        
        grouped_df = df.groupby([self.pid, self.offset])[self.ts_cols].mean() # Use raw value at each time seqeunce.
        result = grouped_df.values.reshape(self.static_df[self.pid].nunique(), self.args.seq_len, len(self.ts_cols))
                
        return result

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return self.len
    

ISTS_EHR_Dataset(args)

In [ ]:
def reshape_data_matrix(self):

    df = self.ts_df.sort_values([self.pid, self.offset]).reset_index(drop=True)
    
    grouped_df = df.groupby([self.pid, self.offset]).mean() # Use raw value at each time seqeunce. 
    result = grouped_df.values.reshape(self.df[self.pid].nunique(), self.args.seq_len, len(self.ts_cols))
            
    return result




In [44]:
df = dynamics_df[dynamics_df['var_name'].isin(vital_cols)].reset_index(drop=True)

grouped_df = df.groupby(['stay_id', 'charttime'])

In [48]:
df

,stay_id,charttime,var_name,value
0,2,5,resp,25.000000
1,2,5,temp,36.799999
2,2,17,hr,55.000000
3,2,17,o2sat,94.000000
4,2,18,hr,59.000000
...,...,...,...,...
55298726,33904,1439,map,66.000000
55298727,33904,1439,sbp,91.000000
55298728,33904,1440,hr,85.000000
55298729,33904,1440,resp,23.900000


In [15]:
static_df.var_name.unique()

array(['age', 'sex', 'height', 'weight'], dtype=object)

In [55]:
# 해당 데이터셋마다 필요한 값들을 excel 형식으로 변환해보자.

def select_vars(db_name, var_list, base_dict):
    """
    min, max => 이상치 처리 시 사용
    cateogries => 해당 변수 category 분류 시 사용
    source_table => 어느 테이블에서 가져와야 하는지
    source_columns => 해당 테이블 어느 컬럼에서 가져와야 하는지
    source_itemid => 해당 테이블의 컬럼에서 어떤 id로 존재하는지
    """
    var_normal_min = []
    var_normal_max = []
    var_unit = []
    source_category = []
    source_table = []
    source_column = []
    source_itemid = []
    source_callback = []
    source_vars = [] # to map other lists
    for var in tqdm(var_list):

        for i in range(len(base_dict[var]['sources'][db_name])): # 여러 테이블에 분산되어 있는 경우 길이가 2가 넘을 수 있기 때문

            source_table.append(base_dict[var]['sources'][db_name][i]['table'])

            try:
                source_column.append(base_dict[var]['sources'][db_name][i]['sub_var'])
            except:
                source_column.append(base_dict[var]['sources'][db_name][i]['val_var'])

            try:
                if [223761, 224027] == base_dict[var]['sources'][db_name][i]['ids']: # temp 관련 변수에서 skin temp는 제외 
                    source_itemid.append(223761)
                
                else:
                    source_itemid.append(base_dict[var]['sources'][db_name][i]['ids'])
            except:
                source_itemid.append(None)

            try:
                source_callback.append(base_dict[var]['sources'][db_name][i]['callback'])
            except:
                source_callback.append(None)


            source_category.append(base_dict[var]['category'])

            try:
                var_unit.append(base_dict[var]['unit'])
            except:
                var_unit.append(None)

            try:
                var_normal_min.append(base_dict[var]['min'])
            except:
                var_normal_min.append(None)
            
            try:
                var_normal_max.append(base_dict[var]['max'])
            except:
                var_normal_max.append(None)

            source_vars.append(var)
        
    return var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars

var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars  = select_vars('miiv', static_vars + dynamic_vars, concept_dict)

MAPPING_DF = pd.DataFrame({
    'var_name' : source_vars,
    'normal_min' : var_normal_min,
    'normal_max' : var_normal_max,
    'category' : source_category,
    'table' : source_table,
    'column' : source_column,
    'itemid' : source_itemid,
    'unit' : var_unit,
    'method' : source_callback 
})

# FUll name 및 추가 
full_name_dict = {'age' : 'Age', 'sex' : 'Sex', 'height' : 'Height', 'weight' : 'Weight', 'sbp' : 'Blood pressure (systolic)', 'dbp' : 'Blood pressure (diastolic)', 'hr' : 'Heart rate', 'map' : 'Mean arterial pressure', 'o2sat' : 'Oxygen saturation',
                  'resp' : 'Respiratory rate', 'temp' : 'Temperature', 'alb' : 'Albumin', 'alp' : 'Alkaline phosphatase', 'alt' : 'Alanine aminotransferase', 'ast' : 'Aspartate aminotransferase', 'be' : 'Base excess', 'bicar' : 'Bicarbonate',
                  'bili' : 'Bilirubin (total)', 'bili_dir' : 'Bilirubin (direct)', 'bnd' : 'Band form neutrophils', 'bun' : 'Blood urea nitrogen', 'ca' : 'Calcium', 'cai' : 'Calcium ionized', 'crea' : 'Creatinine', 'ck' : 'Creatinine kinase',
                  'ckmb' : 'Creatinine kinase MB', 'cl' : 'Chloride', 'pco2' : 'CO2 partial pressure', 'crp' : 'C-reactive protein', 'fgn' : 'Fibrinogen', 'glu' : 'Glucose', 'hgb' : 'Haemoglobin', 'inr_pt' : 'International normalised ratio (INR)',
                  'lact' : 'Lactate', 'lymph' : 'Lymphocytes', 'mch' : 'Mean cell haemoglobin', 'mchc' : 'Mean corpuscular haemoglobin concentration', 'mcv' : 'Mean corpuscular volume', 'methb' : 'Methaemoglobin', 'mg' : 'Magnesium', 
                  'neut' : 'Neutrophils', 'po2' : 'O2 partial pressure', 'ptt' : 'Partial thromboplastin time', 'ph' : 'pH of blood', 'phos' : 'Phosphate', 'plt' : 'Platelets', 'k' : 'Potassium', 'na' : 'Sodium', 'tnt' : 'Troponin T', 
                  'wbc' : 'White blood cells', 'fio2' : 'Fraction of inspired oxygen', 'urine' : 'Urine output'
                  }

fixed_unit_dict = {'age' : 'Years', 'sex' : '', 'height' : 'cm', 'weight' : 'kg', 'sbp' : 'mmHg', 'dbp' : 'mmHg', 'hr' : 'beats/minute', 'map' : 'mmHg', 'o2sat' : '%',
                  'resp' : 'breaths/minute', 'temp' : '°C', 'alb' : 'g/dL', 'alp' : 'IU/L', 'alt' : 'IU/L', 'ast' : 'IU/L', 'be' : 'mmol/L', 'bicar' : 'mmol/L',
                  'bili' : 'mg/dL', 'bili_dir' : 'mg/dL', 'bnd' : '%', 'bun' : 'mg/dL', 'ca' : 'mg/dL', 'cai' : 'mmol/L', 'crea' : 'mg/dL', 'ck' : 'IU/L',
                  'ckmb' : 'ng/mL', 'cl' : 'mmol/L', 'pco2' : 'mmHg', 'crp' : 'mg/L', 'fgn' : 'mg/dL', 'glu' : 'mg/dL', 'hgb' : 'g/dL', 'inr_pt' : '',
                  'lact' : 'mmol/L', 'lymph' : '%', 'mch' : 'pg', 'mchc' : '%', 'mcv' : 'fL', 'methb' : '%', 'mg' : 'mg/dL', 
                  'neut' : '%', 'po2' : 'mmHg', 'ptt' : 'sec', 'ph' : '', 'phos' : 'mg/dL', 'plt' : '1,000 / μL', 'k' : 'mmol/L', 'na' : 'mmol/L', 'tnt' : 'ng/mL', 
                  'wbc' : '1,000 / μL', 'fio2' : '%', 'urine' : 'mL'
                  }

MAPPING_DF['full_var_name'] = MAPPING_DF['var_name'].apply(lambda x : full_name_dict[x])
MAPPING_DF['fixed_unit'] = MAPPING_DF['var_name'].apply(lambda x : fixed_unit_dict[x])
MAPPING_DF = MAPPING_DF[['var_name', 'normal_min', 'normal_max', 'full_var_name', 'fixed_unit']] # only using variables
MAPPING_DF = MAPPING_DF.drop_duplicates().reset_index(drop=True)

100%|██████████| 52/52 [00:00<?, ?it/s]


In [10]:
# lab vars 변경
lab_vars = [full_name_dict[i] for i in lab_vars]
print(len(lab_vars))

40


In [11]:
lab_vars

['Albumin',
 'Alkaline phosphatase',
 'Alanine aminotransferase',
 'Aspartate aminotransferase',
 'Base excess',
 'Bicarbonate',
 'Bilirubin (total)',
 'Bilirubin (direct)',
 'Band form neutrophils',
 'Blood urea nitrogen',
 'Calcium',
 'Calcium ionized',
 'Creatinine kinase',
 'Creatinine kinase MB',
 'Chloride',
 'Creatinine',
 'C-reactive protein',
 'Fibrinogen',
 'Fraction of inspired oxygen',
 'Glucose',
 'Haemoglobin',
 'International normalised ratio (INR)',
 'Potassium',
 'Lactate',
 'Lymphocytes',
 'Mean cell haemoglobin',
 'Mean corpuscular haemoglobin concentration',
 'Mean corpuscular volume',
 'Methaemoglobin',
 'Magnesium',
 'Sodium',
 'Neutrophils',
 'CO2 partial pressure',
 'pH of blood',
 'Phosphate',
 'Platelets',
 'O2 partial pressure',
 'Partial thromboplastin time',
 'Troponin T',
 'White blood cells']

In [6]:
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, Set, Any
import numpy as np
import pandas as pd


# ============================================================
# Config
# ============================================================
@dataclass
class MarkdownSummaryConfig:
    # --- Input dataframe column names ---
    # charttime is already "minutes since admission" (int)
    time_col: str = "charttime"     # minutes since ICU admission (int)
    id_col: str = "stay_id"
    var_col: str = "full_var_name"
    val_col: str = "value"
    unit_col: str = "fixed_unit"
    normal_min : str = 'normal_min'
    normal_max : str = 'normal_max'
    
    # --- Feature definitions ---
    # slope = (Δvalue) / (Δminutes)
    # variability = abs(Δvalue) by default
    variability_mode: str = "abs_delta"  # "abs_delta" or "rolling_std"
    rolling_window: int = 5              # used when variability_mode == "rolling_std"
    min_points_for_stats: int = 2        # rolling/std 최소 관측 수

    # --- Formatting ---
    round_ndigits: int = 1


# ============================================================
# Formatting helpers
# ============================================================
def fmt_number(x: Any, ndigits: int = 1, type='measurement') -> str:
    """Format numbers; use 'Not observed' for None/NaN."""
    
    if x is None and type == 'measurement':
        return "Not observed"
    
    if pd.isna(x) and type == 'normal_max':
        return "inf"
    
    if pd.isna(x) and type == 'normal_min':
        return "-inf"

    try:
        if isinstance(x, float) and np.isnan(x):
            return "Not observed"
    except Exception:
        pass

    if isinstance(x, (int, np.integer)):
        return str(int(x))

    try:
        return f"{float(x):.{ndigits}f}"
    except Exception:
        return "Not observed"


def nan_stats(x: np.ndarray) -> Dict[str, Optional[float]]:
    """Return min/max/mean/std on numeric array, ignoring NaNs."""
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    if x.size == 0:
        return {"min": None, "max": None, 'median' : None ,"mean": None, "std": None}

    std = float(np.std(x, ddof=1)) if x.size > 1 else 0.0
    return {
        "min": float(np.min(x)),
        "max": float(np.max(x)),
        'median' : float(np.median(x)),
        "mean": float(np.mean(x)),
        "std": std,
    }


# ============================================================
# Feature computation (per variable)
# ============================================================
def compute_variable_features(
    var_df: pd.DataFrame,
    cfg: MarkdownSummaryConfig,
) -> Dict[str, Any]:
    """
    Compute summary features for ONE variable group.
    Assumption: cfg.time_col is already minutes (int) since admission.
    """
    df_sorted = var_df.sort_values(cfg.time_col).copy()

    times_min = pd.to_numeric(df_sorted[cfg.time_col], errors="coerce").to_numpy(dtype=float)
    values = pd.to_numeric(df_sorted[cfg.val_col], errors="coerce").to_numpy(dtype=float)

    # observation count (non-NaN values)
    n_obs = int(np.sum(~np.isnan(values)))

    # first/last time (minutes)
    first_min = float(times_min[0]) if times_min.size else None
    last_min = float(times_min[-1]) if times_min.size else None

    # interval mean (unique time 기준)
    unique_times = np.unique(times_min[~np.isnan(times_min)])
    if unique_times.size >= 2:
        interval_mean_min = float(np.mean(np.diff(unique_times)))
    else:
        interval_mean_min = None

    # value stats
    value_stats = nan_stats(values)

    # slope stats: Δy / Δt (dt>0)
    slopes = np.array([], dtype=float)
    if times_min.size >= 2:
        dt = np.diff(times_min)
        dy = np.diff(values)
        mask = (~np.isnan(dt)) & (~np.isnan(dy)) & (dt > 0)
        if np.any(mask):
            slopes = dy[mask] / dt[mask]
    slope_stats = nan_stats(slopes) if slopes.size else {"min": None, "max": None, "mean": None, "std": None}

    # variability stats
    if cfg.variability_mode == "abs_delta":
        variability_seq = np.abs(np.diff(values))
        variability_seq = variability_seq[~np.isnan(variability_seq)]
    elif cfg.variability_mode == "rolling_std":
        rolling_std = (
            pd.Series(values)
            .rolling(cfg.rolling_window, min_periods=cfg.min_points_for_stats)
            .std()
            .to_numpy(dtype=float)
        )
        variability_seq = rolling_std[~np.isnan(rolling_std)]
    else:
        raise ValueError("variability_mode must be 'abs_delta' or 'rolling_std'")

    variability_stats = nan_stats(variability_seq) if variability_seq.size else {"min": None, "max": None, "mean": None, "std": None}

    return {
        "first_min": first_min,
        "last_min": last_min,
        "n_obs": n_obs,
        "interval_mean_min": interval_mean_min,
        "value_stats": value_stats,
        "slope_stats": slope_stats,
        "variability_stats": variability_stats,
    }


# ============================================================
# Markdown builder (per patient)
# ============================================================
def build_patient_markdown_summary(
    observations: pd.DataFrame,
    demographics: pd.DataFrame,
    mapping_df : pd.DataFrame,
    cfg: Optional[MarkdownSummaryConfig] = None,
    labs_title: str = "Summary of Lab Results",
    lab_var_list: Optional[list[str]] = None,
    output_var_names: Optional[Set[str]] = ['Urine output'],
) -> str:
    """
    observations: one patient's observation rows
      - must include: cfg.time_col (minutes), cfg.var_col, cfg.val_col
    demographics: {"Age":..., "Gender":..., "Weight":..., "Height":...}
    normal_ranges: {"WBC": (4,11), ...}
    output_var_names: variables to treat as "Output Events" section
    """
    cfg = cfg or MarkdownSummaryConfig()
    output_var_names = output_var_names or set()

    df = observations.copy()
    df[cfg.val_col] = pd.to_numeric(df[cfg.val_col], errors="coerce")
    df[cfg.time_col] = pd.to_numeric(df[cfg.time_col], errors="coerce")  # ensure numeric minutes

    # ----------------------------
    # Header: demographics
    # ----------------------------
    md_lines = []
    md_lines.append("# Patient Demographics at ICU Admission\n")
    md_lines.append(f"- Age : {fmt_number(demographics[demographics['full_var_name'] == 'Age']['value'].values[0])}")
    md_lines.append(f"- Gender : {demographics[demographics['full_var_name'] == 'Sex']['value'].values[0]}")

    try:
        md_lines.append(f"- Weight : {fmt_number(demographics[demographics['full_var_name'] == 'Weight']['value'].values[0])}kg")
    except:
        md_lines.append(f"- Weight : Not observed")
    
    try:
        md_lines.append(f"- Height : {fmt_number(demographics[demographics['full_var_name'] == 'Height']['value'].values[0])}cm \n")
    except:
        md_lines.append(f"- Height : Not observed")

    md_lines.append("")

    # ----------------------------
    # Split labs vs outputs
    # ----------------------------
    is_output = df[cfg.var_col].isin(output_var_names)
    labs_df = df[~is_output].copy()
    output_df = df[is_output].copy()


    mapping_key = cfg.var_col  # "full_var_name"

    meta_cols = [cfg.unit_col, cfg.normal_min, cfg.normal_max]
    meta_cols = [c for c in meta_cols if c in mapping_df.columns]

    var_meta = (
        mapping_df.drop_duplicates(subset=[mapping_key])
                .set_index(mapping_key)[meta_cols]
                .to_dict(orient="index"))
    
    def render_variable_section(title: str, section_df: pd.DataFrame, var_list: list[str]) -> list[str]:
        section_lines = [f"# {title}"]

        for var_name in var_list:
            # 1) metadata from MappingDF (unit + normal range)
            meta = var_meta.get(var_name, {})
            unit_str = meta.get(cfg.unit_col, "") if meta else ""
            ref_min = meta.get(cfg.normal_min, None) if meta else None
            ref_max = meta.get(cfg.normal_max, None) if meta else None

            # 2) patient rows for this var (may be empty)
            var_rows = section_df[section_df[cfg.var_col] == var_name]

            section_lines.append(f"- [{var_name}] ({unit_str})\n")
            section_lines.append(
                f"\t- Normal Value Range : {fmt_number(ref_min, cfg.round_ndigits, 'normal_min')} to {fmt_number(ref_max, cfg.round_ndigits, 'normal_max')}"
            )

            # 3) if no observations -> keep unit/range, but stats/time are Not observed
            n_valid = int(var_rows[cfg.val_col].notna().sum()) if not var_rows.empty else 0
            if n_valid == 0:
                section_lines.append("\t- (Not observed)")
                continue

            # 4) has observations -> compute features
            feats = compute_variable_features(var_rows, cfg)
            section_lines.append(
                f"\t- First/Last Obs : {fmt_number(feats['first_min'], 0)} / {fmt_number(feats['last_min'], 0)}, "
                f"N : {fmt_number(feats['n_obs'], 0)}, Interval : {fmt_number(feats['interval_mean_min'], 0)}"
            )

            vs = feats["value_stats"]
            section_lines.append(
                "\t- Statistics: "
                f"[{fmt_number(vs['min'], cfg.round_ndigits)}, {fmt_number(vs['max'], cfg.round_ndigits)}, "
                f"{fmt_number(vs['median'], cfg.round_ndigits)}, {fmt_number(vs['mean'], cfg.round_ndigits)}, "
                f"{fmt_number(vs['std'], cfg.round_ndigits)}]"
            )
            section_lines.append("")

        return section_lines


    # def render_variable_section(title: str, section_df: pd.DataFrame, var_list : list = None) -> list[str]:
    #     section_lines = [f"# {title}"]
    #     section_lines.append(f"## Time unit : minutes after admission, Obs : Observation Time, N : observation count, Interval : mean interval between observations, Statistics : [min, max, median, mean, std]")

    #     if section_df.empty:
    #         section_lines.append("- (Not observed)\n")
    #         return section_lines

    #     for var_name, var_df in section_df.groupby(cfg.var_col):
    #         # unit
    #         unit_str = ""
    #         if cfg.unit_col in var_df.columns and var_df[cfg.unit_col].notna().any():
    #             unit_str = str(var_df[cfg.unit_col].dropna().iloc[0])

    #         # normal range
    #         ref_min, ref_max = var_df[cfg.normal_min].unique(), var_df[cfg.normal_max].unique()

    #         # features
    #         feats = compute_variable_features(var_df, cfg)

    #         # (옵션) slope/variability level rule 넣고 싶으면 여기에 정의
    #         slope_level = "None"
    #         variability_level = "None"

    #         section_lines.append(f"- [{var_name}] ({unit_str})\n")
    #         section_lines.append(
    #             f"\t- Normal Value Range : {fmt_number(ref_min, cfg.round_ndigits)} to {fmt_number(ref_max, cfg.round_ndigits)}"
    #         )
    #         section_lines.append(f"\t- First/Last Obs : {fmt_number(feats['first_min'], 0)} / {fmt_number(feats['last_min'], 0)}," \
    #                              f" N : {fmt_number(feats['n_obs'], 0)}, Interval : {fmt_number(feats['interval_mean_min'], 0)}")

    #         vs = feats["value_stats"]
    #         section_lines.append(
    #             "\t- Statistics: "
    #             # f"Min = {fmt_number(vs['min'], cfg.round_ndigits)}, "
    #             # f"Max = {fmt_number(vs['max'], cfg.round_ndigits)}, "
    #             # f"Median = {fmt_number(vs['median'], cfg.round_ndigits)}, "
    #             # f"Mean = {fmt_number(vs['mean'], cfg.round_ndigits)}, "
    #             # f"STD = {fmt_number(vs['std'], cfg.round_ndigits)}"
    #             f"[{fmt_number(vs['min'], cfg.round_ndigits)}, {fmt_number(vs['max'], cfg.round_ndigits)}, {fmt_number(vs['median'], cfg.round_ndigits)}, {fmt_number(vs['mean'], cfg.round_ndigits)}, {fmt_number(vs['std'], cfg.round_ndigits)}]"
    #         )

    #         # ss = feats["slope_stats"]
    #         # section_lines.append(
    #         #     "\t- Statistics of Slopes : "
    #         #     f"Min = {fmt_number(ss['min'], cfg.round_ndigits)} ({slope_level}), "
    #         #     f"Max = {fmt_number(ss['max'], cfg.round_ndigits)} ({slope_level}), "
    #         #     f"Mean = {fmt_number(ss['mean'], cfg.round_ndigits)} ({slope_level}), "
    #         #     f"STD = {fmt_number(ss['std'], cfg.round_ndigits)}"
    #         # )

    #         # vb = feats["variability_stats"]
    #         # section_lines.append(
    #         #     "\t- Statistics of Variability : "
    #         #     f"Min = {fmt_number(vb['min'], cfg.round_ndigits)} ({variability_level}), "
    #         #     f"Max = {fmt_number(vb['max'], cfg.round_ndigits)} ({variability_level}), "
    #         #     f"Mean = {fmt_number(vb['mean'], cfg.round_ndigits)} ({variability_level}), "
    #         #     f"STD = {fmt_number(vb['std'], cfg.round_ndigits)}"
    #         # )
    #         section_lines.append("")  # blank line between vars

    #     return section_lines

    
    md_lines.append("## Time unit : minutes after admission, Obs : Observation Time, N : observation count, "
            "Interval : mean interval between observations, Statistics : [min, max, median, mean, std]")
    md_lines += render_variable_section(labs_title, labs_df, lab_var_list)
    md_lines.append("")  # spacer
    md_lines += render_variable_section("Summary of Output Events", output_df, ['Urine output'])

    return "\n".join(md_lines).strip()


# ============================================================
# Example Usage
# ============================================================
if __name__ == "__main__":


    cfg = MarkdownSummaryConfig(
        time_col="charttime",        # already minutes (int)
        variability_mode="abs_delta",
        rolling_window=5,
        round_ndigits=1
    )

    stay_ids = []
    texts = []

    for sid in tqdm(static_df.stay_id.unique()):
        # change
        patient_df = dynamics_df[(dynamics_df['stay_id'] == sid)]
        patient_df = pd.merge(patient_df, MAPPING_DF, on = 'var_name', how = 'left')

        demographics = static_df[static_df['stay_id'] == sid]
        demographics = pd.merge(demographics, MAPPING_DF, on = 'var_name', how = 'left')


        md_text = build_patient_markdown_summary(
            observations=patient_df,
            demographics=demographics,
            mapping_df=MAPPING_DF,
            cfg=cfg,
            lab_var_list = lab_vars
        )

        stay_ids.append(sid)
        texts.append(md_text)

100%|██████████| 13109/13109 [16:41<00:00, 13.09it/s]


In [ ]:
text_df = pd.DataFrame(columns=['stay_id', 'text'])
text_df['stay_id'] = stay_ids
text_df['text'] = texts

text_df.to_csv(HIRID_ROOT_DIR/'text_df.csv.gz', compression='gzip', index = False)

# add length columns
text_df['length'] = text_df['text'].apply(lambda x : len(x))

max_length = text_df['length'].max()
max_text = text_df[text_df['length'] == max_length]['text'].values[0]
print(max_text)

In [11]:
with open("example.txt", "w", encoding="utf-8") as f:
    f.write(max_text)

In [21]:
with open("example.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text)

# Patient Demographics at ICU Admission

- Age : 71.0
- Gender : Male
- Weight : 94.0kg
- Height : 175.0cm 


## Time unit : minutes after admission, Obs : Observation Time, N : observation count, Interval : mean interval between observations, Statistics : [min, max, median, mean, std]
# Summary of Lab Results
- [Albumin] (g/dL)

	- Normal Value Range : 0.0 to 6.0
	- First/Last Obs : 1333 / 1333, N : 1, Interval : Not observed
	- Statistics: [2.8, 2.8, 2.8, 2.8, 0.0]

- [Alkaline phosphatase] (IU/L)

	- Normal Value Range : 0.0 to inf
	- First/Last Obs : 1333 / 1333, N : 1, Interval : Not observed
	- Statistics: [37.0, 37.0, 37.0, 37.0, 0.0]

- [Alanine aminotransferase] (IU/L)

	- Normal Value Range : 0.0 to inf
	- First/Last Obs : 1333 / 1333, N : 1, Interval : Not observed
	- Statistics: [141.0, 141.0, 141.0, 141.0, 0.0]

- [Aspartate aminotransferase] (IU/L)

	- Normal Value Range : 0.0 to inf
	- First/Last Obs : 1333 / 1333, N : 1, Interval : Not observed
	- Statistics: [332.0, 33

In [42]:
# Max text length 확인
mimic_text = pd.read_csv(MIMIC_ROOT_DIR/'text_df.csv.gz', compression='gzip')
eicu_text = pd.read_csv(EICU_ROOT_DIR/'text_df.csv.gz', compression='gzip')
hirid_text = pd.read_csv(HIRID_ROOT_DIR/'text_df.csv.gz', compression='gzip')

mimic_text['length'] = mimic_text['text'].apply(lambda x : len(x))
eicu_text['length'] = eicu_text['text'].apply(lambda x : len(x))
hirid_text['length'] = hirid_text['text'].apply(lambda x : len(x))


In [43]:
max_length = mimic_text['length'].max()
mimic_max_text = mimic_text[mimic_text['length'] == max_length]['text'].values[0]

max_length = eicu_text['length'].max()
eicu_max_text = eicu_text[eicu_text['length'] == max_length]['text'].values[0]

max_length = hirid_text['length'].max()
hirid_max_text = hirid_text[hirid_text['length'] == max_length]['text'].values[0]

## Example for using LLaMA Tokenizer

In [19]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_id = "meta-llama/Llama-3.1-8B" 

config = AutoConfig.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.39s/it]


In [39]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_id = "meta-llama/Llama-3.1-8B" 

config = AutoConfig.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

tokenizer.pad_token = tokenizer.eos_token
# padding='max_length', max_length = 4800


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


torch.Size([1, 3258, 4096])


In [44]:
# mimic
mimic_inputs = tokenizer(mimic_max_text, truncation=True, return_tensors='pt')
mimic_inputs['input_ids']

# device로 이동
mimic_inputs = {k: v.to(model.device) for k, v in mimic_inputs.items()}

with torch.no_grad():
    # input embedding table에서 바로 lookup
    token_embeddings = model.get_input_embeddings()(mimic_inputs["input_ids"])
    # shape: [batch=1, seq_len, hidden_size]
    print(token_embeddings.shape)

# eicu
eicu_inputs = tokenizer(eicu_max_text, truncation=True, return_tensors='pt')
eicu_inputs['input_ids']

# device로 이동
eicu_inputs = {k: v.to(model.device) for k, v in eicu_inputs.items()}

with torch.no_grad():
    # input embedding table에서 바로 lookup
    token_embeddings = model.get_input_embeddings()(eicu_inputs["input_ids"])
    # shape: [batch=1, seq_len, hidden_size]
    print(token_embeddings.shape)

# hirid
hirid_inputs = tokenizer(hirid_max_text, truncation=True, return_tensors='pt')
hirid_inputs['input_ids']

# device로 이동
hirid_inputs = {k: v.to(model.device) for k, v in hirid_inputs.items()}

with torch.no_grad():
    # input embedding table에서 바로 lookup
    token_embeddings = model.get_input_embeddings()(hirid_inputs["input_ids"])
    # shape: [batch=1, seq_len, hidden_size]
    print(token_embeddings.shape)

torch.Size([1, 3277, 4096])
torch.Size([1, 3258, 4096])
torch.Size([1, 3327, 4096])


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.1-8B" 

TOKEN = 'your tokens'
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=f"{TOKEN}")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

enc = tokenizer(
    max_text,
    return_tensors="pt",
    truncation=True,
    max_length=8192,   # 필요 시 조절
)

# device로 이동
enc = {k: v.to(model.device) for k, v in enc.items()}

with torch.no_grad():
    # input embedding table에서 바로 lookup
    token_embeddings = model.get_input_embeddings()(enc["input_ids"])
    # shape: [batch=1, seq_len, hidden_size]

print(token_embeddings.shape)


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.41s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


torch.Size([1, 3327, 4096])


In [14]:
full_ids = tokenizer(max_text, add_special_tokens=False).input_ids
print("full length:", len(full_ids))
print("will be truncated to:", 8192, "?", len(full_ids) > 8192)

full length: 3326
will be truncated to: 8192 ? False


In [15]:
enc = tokenizer(
    md_text,
    return_tensors="pt",
    max_length=4096,
    return_overflowing_tokens=False,
)

ids = enc["input_ids"][0].tolist()                 # [seq_len]
tokens = tokenizer.convert_ids_to_tokens(ids)      # 토큰 문자열(대부분 BPE 조각)

for i, (tid, tok) in enumerate(zip(ids[:200], tokens[:200])):  # 앞 200개만 예시
    print(f"{i:4d}  {tid:8d}  {repr(tok)}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


   0    128000  '<|begin_of_text|>'
   1         2  '#'
   2     30024  'ĠPatient'
   3      4829  'ĠDem'
   4     45245  'ographics'
   5       520  'Ġat'
   6     85015  'ĠICU'
   7     63446  'ĠAdmission'
   8       271  'ĊĊ'
   9        12  '-'
  10     13381  'ĠAge'
  11       551  'Ġ:'
  12       220  'Ġ'
  13      2031  '70'
  14        13  '.'
  15       931  '000'
  16       198  'Ċ'
  17        12  '-'
  18     29317  'ĠGender'
  19       551  'Ġ:'
  20       435  'ĠF'
  21       198  'Ċ'
  22        12  '-'
  23     16923  'ĠWeight'
  24       551  'Ġ:'
  25       220  'Ġ'
  26      1399  '60'
  27        13  '.'
  28       931  '000'
  29      7501  'kg'
  30       198  'Ċ'
  31        12  '-'
  32     22147  'ĠHeight'
  33       551  'Ġ:'
  34       220  'Ġ'
  35      8258  '170'
  36        13  '.'
  37       931  '000'
  38      6358  'cm'
  39     15073  'ĠĊĊĊ'
  40       567  '##'
  41      4212  'ĠTime'
  42      5089  'Ġunit'
  43       551  'Ġ:'
  44      4520  'Ġmi